In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)
np.random.seed(42)

def get_xception_based_model():
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    inputs = tf.keras.Input(shape=(128, 128, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
    outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    opt = Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    img = tf.cast(img, tf.float32) / 255.0
    return img

def predict_with_threshold(model, image_path, threshold=0.5):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)[0][0]

    print(f"Raw Prediction Score: {prediction:.4f}.")

    if prediction >= threshold:
        pred_class = 'real'
    else:
        pred_class = 'fake'

    print(f"Predicted Class ({pred_class}), Probability - {prediction:.4f}.")

In [ ]:

if __name__=="__main__":
    xception_model = get_xception_based_model()

    # Optionally uncomment the next line to visualize the architecture summary
    # xception_model.summary()

    train_datagen = ImageDataGenerator(
      rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      rotation_range=10.,
      width_shift_range=0.1,
      height_shift_range=0.1)

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/training',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    validate_generator = validation_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/test',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    epochs = 20

    history = xception_model.fit(train_generator, epochs=epochs, validation_data=validate_generator)

    xception_model.save('best_xception_model.h5')

    # # Testing
    # image_path = '<YOUR_IMAGE_PATH>'
    # predict_with_threshold(xception_model, image_path)

83683744/83683744 [==============================] - 0s 0us/step
Found 1437 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Epoch 1/20
6/6 [==============================] - 335s 51s/step - loss: 0.7262 - accuracy: 0.4864 - val_loss: 0.6947 - val_accuracy: 0.4719
Epoch 2/20
6/6 [==============================] - 30s 5s/step - loss: 0.6882 - accuracy: 0.5449 - val_loss: 0.6897 - val_accuracy: 0.4967
Epoch 3/20
6/6 [==============================] - 34s 6s/step - loss: 0.6812 - accuracy: 0.5428 - val_loss: 0.6839 - val_accuracy: 0.5199
Epoch 4/20
6/6 [==============================] - 30s 5s/step - loss: 0.6701 - accuracy: 0.5651 - val_loss: 0.6667 - val_accuracy: 0.6407
Epoch 5/20
6/6 [==============================] - 30s 5s/step - loss: 0.6520 - accuracy: 0.6124 - val_loss: 0.6329 - val_accuracy: 0.6838
Epoch 6/20
6/6 [==============================] - 34s 6s/step - loss: 0.6349 - accuracy: 0.6500 - val_loss: 0.6057 - val_accuracy: 0.6854
Epoch 7/20
6/6 [======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
 # Testing
image_path = '/content/drive/MyDrive/dataset/face_pred/check.jpg'
predict_with_threshold(xception_model, image_path)

image_path = '/content/drive/MyDrive/dataset/face_pred/check2.jpg'
predict_with_threshold(xception_model, image_path)

image_path = '/content/drive/MyDrive/dataset/face_pred/check3.jpg'
predict_with_threshold(xception_model, image_path)

image_path = '/content/drive/MyDrive/dataset/face_pred/check4.jpg'
predict_with_threshold(xception_model, image_path)

1/1 [==============================] - 2s 2s/step
Raw Prediction Score: 0.8392.
Predicted Class (real), Probability - 0.8392.
1/1 [==============================] - 0s 21ms/step
Raw Prediction Score: 0.9071.
Predicted Class (real), Probability - 0.9071.
1/1 [==============================] - 0s 21ms/step
Raw Prediction Score: 0.2671.
Predicted Class (fake), Probability - 0.2671.
1/1 [==============================] - 0s 21ms/step
Raw Prediction Score: 0.0937.
Predicted Class (fake), Probability - 0.0937.
